In [1]:
import sympy as sm
import networkx as nx
import matplotlib.pyplot as plt
from sympy.physics.vector import dynamicsymbols

sm.init_printing(pretty_print=False, use_latex='mathjax', forecolor='White')

In [2]:
from matrices import vector ,quatrenion, A

In [9]:
class AbstractMatrix(sm.MatrixExpr):
    """
    **Abstract Class**
    
    Base class of symbolic matrices which their values are evaluated based 
    on given parameters. Can be thought of as an undefind functions that
    returns a matrix

    Class Attributes
    ----------------
    is_commutative : False
        
    is_Matrix : True
    
    """
    
    is_commutative = False
    is_Matrix = True
    shape = (3,3)
    
    def __init__(self,*args):
        pass
    
    def _latex(self,expr):
        name = self.__class__.__name__
        return r'{%s%s}'%(name,self.args,)
    
    def _entry(self,i,j, *args):
        v = sm.MatrixSlice(self,i,j)
        return v
    
    
class A(AbstractMatrix):
    """
    Representaion of symbolic transformation matrix which represents the 
    orientation of a rigid body in 3D space. The matrix is a function of 
    the orientation parameters used, e.g. Euler-Parameters.
    
    Parameters
    ----------
    P : quatrenion or vector
        Orientation parameters of the rigid body.
    """
    shape = (3,3)
    
    def __init__(self, P):
        e0, e1, e2, e3 = P.as_explicit()
    
        a00 = (e0**2+e1**2-e2**2-e3**2)
        a01 = 2*((e1*e2)-(e0*e3))              
        a02 = 2*((e1*e3)+(e0*e2))

        a10 = 2*((e1*e2)+(e0*e3))
        a11 = e0**2-e1**2+e2**2-e3**2
        a12 = 2*((e2*e3)-(e0*e1))

        a20 = 2*((e1*e3)-(e0*e2))
        a21 = 2*((e2*e3)+(e0*e1))
        a22 = e0**2-e1**2-e2**2+e3**2

        mat = sm.Matrix([[a00, a01, a02],
                         [a10, a11, a12],
                         [a20, a21, a22]])
        self._data = mat

    def _latex(self,expr):
        p = self.args[0]
        return r'{A(%s)}'%p.name
    
    def _entry(self, i, j, *args):
        v = self._data[i,j]
        return v


In [5]:
from sympy.matrices.expressions.matexpr import MatrixElement
class mat_element(MatrixElement):
    
    @property
    def name(self):
        """
        Returns the formated name of the instance which is suitable for Latex
        printings.
        """
        n = r'{{%s}_{(%s,%s)}}'%(self._args[0].name, self._args[1], self._args[2])
        return n

    def _latex(self,expr):
        return r'{{%s}_{(%s,%s)}}'%(self._args[0].name, self._args[1], self._args[2])
    
    def _sympystr (self,expr):
        return '%s[%s, %s]'%(self._args[0].name, self._args[1], self._args[2])
        
def _entry(self, i, j, **args):
    return mat_element(self, i, j)

vector._entry = _entry
quatrenion._entry = _entry

In [3]:
Ri = vector('R_i', format_as= r'{{R}^{i}}')
Pi = quatrenion('P_i', format_as= r'{{P}^{i}}')
ui = vector('u_i', format_as= r'{{u}^{i}_{1}}')

Rj = vector('R_j', format_as= r'{{R}^{j}}')
Pj = quatrenion('Pji', format_as= r'{{P}^{j}}')
uj = vector('u_j', format_as= r'{{u}^{j}_{2}}')

dij = Ri + A(Pi)*ui - Rj - A(Pj)*uj
dij_exp = dij.as_explicit()

dij_exp

Matrix([
[(2*{{P}^{i}}[0, 0]*{{P}^{i}}[2, 0] + 2*{{P}^{i}}[1, 0]*{{P}^{i}}[3, 0])*{{u}^{i}_{1}}[2, 0] + (-2*{{P}^{i}}[0, 0]*{{P}^{i}}[3, 0] + 2*{{P}^{i}}[1, 0]*{{P}^{i}}[2, 0])*{{u}^{i}_{1}}[1, 0] - (2*{{P}^{j}}[0, 0]*{{P}^{j}}[2, 0] + 2*{{P}^{j}}[1, 0]*{{P}^{j}}[3, 0])*{{u}^{j}_{2}}[2, 0] - (-2*{{P}^{j}}[0, 0]*{{P}^{j}}[3, 0] + 2*{{P}^{j}}[1, 0]*{{P}^{j}}[2, 0])*{{u}^{j}_{2}}[1, 0] + ({{P}^{i}}[0, 0]**2 + {{P}^{i}}[1, 0]**2 - {{P}^{i}}[2, 0]**2 - {{P}^{i}}[3, 0]**2)*{{u}^{i}_{1}}[0, 0] - ({{P}^{j}}[0, 0]**2 + {{P}^{j}}[1, 0]**2 - {{P}^{j}}[2, 0]**2 - {{P}^{j}}[3, 0]**2)*{{u}^{j}_{2}}[0, 0] + {{R}^{i}}[0, 0] - {{R}^{j}}[0, 0]],
[(-2*{{P}^{i}}[0, 0]*{{P}^{i}}[1, 0] + 2*{{P}^{i}}[2, 0]*{{P}^{i}}[3, 0])*{{u}^{i}_{1}}[2, 0] + (2*{{P}^{i}}[0, 0]*{{P}^{i}}[3, 0] + 2*{{P}^{i}}[1, 0]*{{P}^{i}}[2, 0])*{{u}^{i}_{1}}[0, 0] - (-2*{{P}^{j}}[0, 0]*{{P}^{j}}[1, 0] + 2*{{P}^{j}}[2, 0]*{{P}^{j}}[3, 0])*{{u}^{j}_{2}}[2, 0] - (2*{{P}^{j}}[0, 0]*{{P}^{j}}[3, 0] + 2*{{P}^{j}}[1, 0]*{{P}^{j}}[2, 0])*{{u}^{j

In [4]:
states = sm.Matrix([*Ri.as_explicit(), *Pi.as_explicit(), *Rj.as_explicit(), *Pj.as_explicit()])
states

Matrix([
[{{R}^{i}}[0, 0]],
[{{R}^{i}}[1, 0]],
[{{R}^{i}}[2, 0]],
[{{P}^{i}}[0, 0]],
[{{P}^{i}}[1, 0]],
[{{P}^{i}}[2, 0]],
[{{P}^{i}}[3, 0]],
[{{R}^{j}}[0, 0]],
[{{R}^{j}}[1, 0]],
[{{R}^{j}}[2, 0]],
[{{P}^{j}}[0, 0]],
[{{P}^{j}}[1, 0]],
[{{P}^{j}}[2, 0]],
[{{P}^{j}}[3, 0]]])

In [5]:
dij_exp.jacobian(states)

Matrix([
[1, 0, 0, 2*{{P}^{i}}[0, 0]*{{u}^{i}_{1}}[0, 0] + 2*{{P}^{i}}[2, 0]*{{u}^{i}_{1}}[2, 0] - 2*{{P}^{i}}[3, 0]*{{u}^{i}_{1}}[1, 0],  2*{{P}^{i}}[1, 0]*{{u}^{i}_{1}}[0, 0] + 2*{{P}^{i}}[2, 0]*{{u}^{i}_{1}}[1, 0] + 2*{{P}^{i}}[3, 0]*{{u}^{i}_{1}}[2, 0],  2*{{P}^{i}}[0, 0]*{{u}^{i}_{1}}[2, 0] + 2*{{P}^{i}}[1, 0]*{{u}^{i}_{1}}[1, 0] - 2*{{P}^{i}}[2, 0]*{{u}^{i}_{1}}[0, 0], -2*{{P}^{i}}[0, 0]*{{u}^{i}_{1}}[1, 0] + 2*{{P}^{i}}[1, 0]*{{u}^{i}_{1}}[2, 0] - 2*{{P}^{i}}[3, 0]*{{u}^{i}_{1}}[0, 0], -1,  0,  0, -2*{{P}^{j}}[0, 0]*{{u}^{j}_{2}}[0, 0] - 2*{{P}^{j}}[2, 0]*{{u}^{j}_{2}}[2, 0] + 2*{{P}^{j}}[3, 0]*{{u}^{j}_{2}}[1, 0], -2*{{P}^{j}}[1, 0]*{{u}^{j}_{2}}[0, 0] - 2*{{P}^{j}}[2, 0]*{{u}^{j}_{2}}[1, 0] - 2*{{P}^{j}}[3, 0]*{{u}^{j}_{2}}[2, 0], -2*{{P}^{j}}[0, 0]*{{u}^{j}_{2}}[2, 0] - 2*{{P}^{j}}[1, 0]*{{u}^{j}_{2}}[1, 0] + 2*{{P}^{j}}[2, 0]*{{u}^{j}_{2}}[0, 0],  2*{{P}^{j}}[0, 0]*{{u}^{j}_{2}}[1, 0] - 2*{{P}^{j}}[1, 0]*{{u}^{j}_{2}}[2, 0] + 2*{{P}^{j}}[3, 0]*{{u}^{j}_{2}}[0, 0]],
[0, 1, 0,

In [6]:
def Der(i):
    return sm.Derivative(i, 't')

states.applyfunc(Der)

Matrix([
[Derivative({{R}^{i}}[0, 0], t)],
[Derivative({{R}^{i}}[1, 0], t)],
[Derivative({{R}^{i}}[2, 0], t)],
[Derivative({{P}^{i}}[0, 0], t)],
[Derivative({{P}^{i}}[1, 0], t)],
[Derivative({{P}^{i}}[2, 0], t)],
[Derivative({{P}^{i}}[3, 0], t)],
[Derivative({{R}^{j}}[0, 0], t)],
[Derivative({{R}^{j}}[1, 0], t)],
[Derivative({{R}^{j}}[2, 0], t)],
[Derivative({{P}^{j}}[0, 0], t)],
[Derivative({{P}^{j}}[1, 0], t)],
[Derivative({{P}^{j}}[2, 0], t)],
[Derivative({{P}^{j}}[3, 0], t)]])

In [7]:
sm.cse(dij_exp.jacobian(states))[1][0]

Matrix([
[1, 0, 0, x11,              x13,           x17, -x18 + x19 - x20, -1,  0,  0, x34,             x36,             x42, x43 - x44 + x45],
[0, 1, 0, x46, -x14 - x15 + x16,           x13,              x11,  0, -1,  0, x47, x38 + x40 - x41,             x36,             x34],
[0, 0, 1, x17,              x46, x10 - x4 - x7,              x13,  0,  0, -1, x42,             x47, x25 + x29 - x33,             x36]])

In [18]:
dij_exp.applyfunc(Der)

Matrix([
[Derivative((2*{{P}^{i}}[0, 0]*{{P}^{i}}[2, 0] + 2*{{P}^{i}}[1, 0]*{{P}^{i}}[3, 0])*{{u}^{i}_{1}}[2, 0] + (-2*{{P}^{i}}[0, 0]*{{P}^{i}}[3, 0] + 2*{{P}^{i}}[1, 0]*{{P}^{i}}[2, 0])*{{u}^{i}_{1}}[1, 0] - (2*{{P}^{j}}[0, 0]*{{P}^{j}}[2, 0] + 2*{{P}^{j}}[1, 0]*{{P}^{j}}[3, 0])*{{u}^{j}_{2}}[2, 0] - (-2*{{P}^{j}}[0, 0]*{{P}^{j}}[3, 0] + 2*{{P}^{j}}[1, 0]*{{P}^{j}}[2, 0])*{{u}^{j}_{2}}[1, 0] + ({{P}^{i}}[0, 0]**2 + {{P}^{i}}[1, 0]**2 - {{P}^{i}}[2, 0]**2 - {{P}^{i}}[3, 0]**2)*{{u}^{i}_{1}}[0, 0] - ({{P}^{j}}[0, 0]**2 + {{P}^{j}}[1, 0]**2 - {{P}^{j}}[2, 0]**2 - {{P}^{j}}[3, 0]**2)*{{u}^{j}_{2}}[0, 0] + {{R}^{i}}[0, 0] - {{R}^{j}}[0, 0], t)],
[Derivative((-2*{{P}^{i}}[0, 0]*{{P}^{i}}[1, 0] + 2*{{P}^{i}}[2, 0]*{{P}^{i}}[3, 0])*{{u}^{i}_{1}}[2, 0] + (2*{{P}^{i}}[0, 0]*{{P}^{i}}[3, 0] + 2*{{P}^{i}}[1, 0]*{{P}^{i}}[2, 0])*{{u}^{i}_{1}}[0, 0] - (-2*{{P}^{j}}[0, 0]*{{P}^{j}}[1, 0] + 2*{{P}^{j}}[2, 0]*{{P}^{j}}[3, 0])*{{u}^{j}_{2}}[2, 0] - (2*{{P}^{j}}[0, 0]*{{P}^{j}}[3, 0] + 2*{{P}^{j}}[1, 0

In [22]:
sm.linear_eq_to_matrix(dij_exp.applyfunc(Der), *[sm.Symbol(i) for i in states.applyfunc(Der)])

TypeError: name should be a string, not <class 'sympy.core.function.Derivative'>

In [20]:
sm.Derivative(dij_exp, 't')

Derivative(Matrix([
[(2*{{P}^{i}}[0, 0]*{{P}^{i}}[2, 0] + 2*{{P}^{i}}[1, 0]*{{P}^{i}}[3, 0])*{{u}^{i}_{1}}[2, 0] + (-2*{{P}^{i}}[0, 0]*{{P}^{i}}[3, 0] + 2*{{P}^{i}}[1, 0]*{{P}^{i}}[2, 0])*{{u}^{i}_{1}}[1, 0] - (2*{{P}^{j}}[0, 0]*{{P}^{j}}[2, 0] + 2*{{P}^{j}}[1, 0]*{{P}^{j}}[3, 0])*{{u}^{j}_{2}}[2, 0] - (-2*{{P}^{j}}[0, 0]*{{P}^{j}}[3, 0] + 2*{{P}^{j}}[1, 0]*{{P}^{j}}[2, 0])*{{u}^{j}_{2}}[1, 0] + ({{P}^{i}}[0, 0]**2 + {{P}^{i}}[1, 0]**2 - {{P}^{i}}[2, 0]**2 - {{P}^{i}}[3, 0]**2)*{{u}^{i}_{1}}[0, 0] - ({{P}^{j}}[0, 0]**2 + {{P}^{j}}[1, 0]**2 - {{P}^{j}}[2, 0]**2 - {{P}^{j}}[3, 0]**2)*{{u}^{j}_{2}}[0, 0] + {{R}^{i}}[0, 0] - {{R}^{j}}[0, 0]],
[(-2*{{P}^{i}}[0, 0]*{{P}^{i}}[1, 0] + 2*{{P}^{i}}[2, 0]*{{P}^{i}}[3, 0])*{{u}^{i}_{1}}[2, 0] + (2*{{P}^{i}}[0, 0]*{{P}^{i}}[3, 0] + 2*{{P}^{i}}[1, 0]*{{P}^{i}}[2, 0])*{{u}^{i}_{1}}[0, 0] - (-2*{{P}^{j}}[0, 0]*{{P}^{j}}[1, 0] + 2*{{P}^{j}}[2, 0]*{{P}^{j}}[3, 0])*{{u}^{j}_{2}}[2, 0] - (2*{{P}^{j}}[0, 0]*{{P}^{j}}[3, 0] + 2*{{P}^{j}}[1, 0]*{{P}^{j}}[2, 

In [19]:
sm.Derivative(Ri.as_explicit(), 't')

Derivative(Matrix([
[{{R}^{i}}[0, 0]],
[{{R}^{i}}[1, 0]],
[{{R}^{i}}[2, 0]]]), t)

In [13]:
sm.Derivative??

Init signature: sm.Derivative(expr, *variables, **kwargs)
Source:        
class Derivative(Expr):
    """
    Carries out differentiation of the given expression with respect to symbols.

    Examples

    >>> from sympy import Derivative, Function, symbols, Subs
    >>> from sympy.abc import x, y
    >>> f, g = symbols('f g', cls=Function)

    >>> Derivative(x**2, x, evaluate=True)
    2*x

    Denesting of derivatives retains the ordering of variables:

        >>> Derivative(Derivative(f(x, y), y), x)
        Derivative(f(x, y), y, x)

    Contiguously identical symbols are merged into a tuple giving
    the symbol and the count:

        >>> Derivative(f(x), x, x, y, x)
        Derivative(f(x), (x, 2), y, x)

    If the derivative cannot be performed, and evaluate is True, the
    order of the variables of differentiation will be made canonical:

        >>> Derivative(f(x, y), y, x, evaluate=True)
        Derivative(f(x, y), x, y)

    Derivatives with respect to undefined functi

In [8]:
Ri.diff('t', 2)

2

In [14]:
vsym = sm.MatrixSymbol('v', 3, 1)

In [18]:
vsym._eval_derivative??

Signature: vsym._eval_derivative(x)
Docstring: <no docstring>
Source:   
    def _eval_derivative(self, x):
        return _matrix_derivative(self, x)
File:      c:\users\ali.mohamed\appdata\local\continuum\anaconda3\lib\site-packages\sympy\matrices\expressions\matexpr.py
Type:      method


In [113]:
class mod_symbol(sm.Symbol):
    
    '''def __new__(cls, name, level=0, **assumptions):

        cls._sanitize(assumptions, cls)
        obj = sm.Symbol.__xnew__(cls, name, **assumptions)
        
        obj._func_symbol = sm.Function(name)('t')
        obj.level = level
        
        return obj'''
    
    def diff(self, *symbols, **assumptions):
        #assumptions.setdefault("evaluate", True)
        return sm.Derivative(self, *symbols, **assumptions)

In [114]:
x = mod_symbol('x')

In [115]:
x2 = x.diff('t')
x2

Derivative(x, t)

In [122]:
x2.args

(x, (t, 1))

In [116]:
sm.Derivative(x2, 't')

Derivative(x, (t, 2))

In [117]:
print(sm.Derivative(x, 'p'))

Derivative(x, p)


In [120]:
f = sm.symbols('f')

In [17]:
symbols = dynamicsymbols

In [2]:
def skew(v):
    x, y, z = v
    vs = sm.Matrix([[0, -z, y], [z, 0, -x], [-y, x, 0]])
    return vs

def A(p):
    e0, e1, e2, e3 = p
    
    a00 = (e0**2+e1**2-e2**2-e3**2)
    a01 = 2*((e1*e2)-(e0*e3))              
    a02 = 2*((e1*e3)+(e0*e2))
    
    a10 = 2*((e1*e2)+(e0*e3))
    a11 = e0**2-e1**2+e2**2-e3**2
    a12 = 2*((e2*e3)-(e0*e1))
    
    a20 = 2*((e1*e3)-(e0*e2))
    a21 = 2*((e2*e3)+(e0*e1))
    a22 = e0**2-e1**2-e2**2+e3**2
    
    mat = sm.Matrix([[a00, a01, a02],
                     [a10, a11, a12],
                     [a20, a21, a22]])
    return mat


def B(p, u):
    e0, e1, e2, e3 = p
    ux, uy, uz = u
    
    a00 = 2*e0*ux + 2*e2*uz - 2*e3*uy
    a01 = 2*e1*ux + 2*e2*uy + 2*e3*uz
    a02 = 2*e0*uz + 2*e1*uy - 2*e2*ux
    a03 = -2*e0*uy + 2*e1*uz - 2*e3*ux
    
    a10 = 2*e0*uy - 2*e1*uz + 2*e3*ux
    a11 = -2*e0*uz - 2*e1*uy + 2*e2*ux
    a12 = 2*e1*ux + 2*e2*uy + 2*e3*uz
    a13 = 2*e0*ux + 2*e2*uz - 2*e3*uy
    
    a20 = 2*e0*uz + 2*e1*uy - 2*e2*ux
    a21 = 2*e0*uy - 2*e1*uz + 2*e3*ux
    a22 = -2*e0*ux - 2*e2*uz + 2*e3*uy
    a23 = 2*e1*ux + 2*e2*uy + 2*e3*uz
    
    mat = sm.Matrix([[a00, a01, a02, a03],
                     [a10, a11, a12, a13],
                     [a20, a21, a22, a23]])
    
    return mat


def E(p):
    e0, e1, e2, e3 = p
    mat = sm.Matrix([[-e1, e0,-e3, e2],
                     [-e2, e3, e0,-e1],
                     [-e3,-e2, e1, e0]])
    return mat

def G(p):
    e0, e1, e2, e3 = p
    mat = sm.Matrix([[-e1, e0, e3,-e2],
                     [-e2,-e3, e0, e1],
                     [-e3, e2,-e1, e0]])
    return mat

In [24]:
Ri = sm.Matrix(['x_i', 'y_i', 'z_i'])
Pi = sm.Matrix(['e0_i', 'e1_i', 'e2_i', 'e3_i'])
ui = sm.Matrix(['ux_i', 'uy_i', 'uz_i'])

Rj = sm.Matrix(['x_j', 'y_j', 'z_j'])
Pj = sm.Matrix(['e0_j', 'e1_j', 'e2_j', 'e3_j'])
uj = sm.Matrix(['ux_j', 'uy_j', 'uz_j'])

dij = Ri + A(Pi)*ui - Rj - A(Pj)*uj
dij.jacobian(sm.Matrix([*Ri, *Pi]))

Matrix([
[1, 0, 0, 2*e0_i*ux_i + 2*e2_i*uz_i - 2*e3_i*uy_i,  2*e1_i*ux_i + 2*e2_i*uy_i + 2*e3_i*uz_i,  2*e0_i*uz_i + 2*e1_i*uy_i - 2*e2_i*ux_i, -2*e0_i*uy_i + 2*e1_i*uz_i - 2*e3_i*ux_i],
[0, 1, 0, 2*e0_i*uy_i - 2*e1_i*uz_i + 2*e3_i*ux_i, -2*e0_i*uz_i - 2*e1_i*uy_i + 2*e2_i*ux_i,  2*e1_i*ux_i + 2*e2_i*uy_i + 2*e3_i*uz_i,  2*e0_i*ux_i + 2*e2_i*uz_i - 2*e3_i*uy_i],
[0, 0, 1, 2*e0_i*uz_i + 2*e1_i*uy_i - 2*e2_i*ux_i,  2*e0_i*uy_i - 2*e1_i*uz_i + 2*e3_i*ux_i, -2*e0_i*ux_i - 2*e2_i*uz_i + 2*e3_i*uy_i,  2*e1_i*ux_i + 2*e2_i*uy_i + 2*e3_i*uz_i]])

In [27]:
Ri = sm.Matrix(symbols('x_i, y_i, z_i'))
Pi = sm.Matrix(symbols('e0_i, e1_i, e2_i, e3_i'))
ui = sm.Matrix(symbols('ux_i, uy_i, uz_i'))

Rj = sm.Matrix(symbols('x_j, y_j, z_j'))
Pj = sm.Matrix(symbols('e0_j, e1_j, e2_j, e3_j'))
uj = sm.Matrix(symbols('ux_j, uy_j, uz_j'))

dij = Ri + A(Pi)*ui - Rj - A(Pj)*uj
jac_i = dij.jacobian(sm.Matrix([*Ri, *Pi]))
jac_i

Matrix([
[1, 0, 0, 2*e0_i(t)*ux_i(t) + 2*e2_i(t)*uz_i(t) - 2*e3_i(t)*uy_i(t),  2*e1_i(t)*ux_i(t) + 2*e2_i(t)*uy_i(t) + 2*e3_i(t)*uz_i(t),  2*e0_i(t)*uz_i(t) + 2*e1_i(t)*uy_i(t) - 2*e2_i(t)*ux_i(t), -2*e0_i(t)*uy_i(t) + 2*e1_i(t)*uz_i(t) - 2*e3_i(t)*ux_i(t)],
[0, 1, 0, 2*e0_i(t)*uy_i(t) - 2*e1_i(t)*uz_i(t) + 2*e3_i(t)*ux_i(t), -2*e0_i(t)*uz_i(t) - 2*e1_i(t)*uy_i(t) + 2*e2_i(t)*ux_i(t),  2*e1_i(t)*ux_i(t) + 2*e2_i(t)*uy_i(t) + 2*e3_i(t)*uz_i(t),  2*e0_i(t)*ux_i(t) + 2*e2_i(t)*uz_i(t) - 2*e3_i(t)*uy_i(t)],
[0, 0, 1, 2*e0_i(t)*uz_i(t) + 2*e1_i(t)*uy_i(t) - 2*e2_i(t)*ux_i(t),  2*e0_i(t)*uy_i(t) - 2*e1_i(t)*uz_i(t) + 2*e3_i(t)*ux_i(t), -2*e0_i(t)*ux_i(t) - 2*e2_i(t)*uz_i(t) + 2*e3_i(t)*uy_i(t),  2*e1_i(t)*ux_i(t) + 2*e2_i(t)*uy_i(t) + 2*e3_i(t)*uz_i(t)]])

In [30]:
acc = dij.diff('t',2)

In [41]:
syms = 'x_i2, y_i2, z_i2, e0_i2, e1_i2, e2_i2, e3_i2, x_j2, y_j2, z_j2, e0_j2, e1_j2, e2_j2, e3_j2'.split(', ')
subs = dict(zip([*sm.Matrix([*(Ri.diff('t',2)), *(Pi.diff('t',2)), *(Rj.diff('t',2)), *(Pj.diff('t',2))])], syms))
acc = acc.subs(subs)
sys = sm.linear_eq_to_matrix(acc, sm.symbols(syms))

In [45]:
sys[1][0]

-2*(e0_i(t)*e2_i(t) + e1_i(t)*e3_i(t))*Derivative(uz_i(t), (t, 2)) + 2*(e0_i(t)*e3_i(t) - e1_i(t)*e2_i(t))*Derivative(uy_i(t), (t, 2)) + 2*(e0_j(t)*e2_j(t) + e1_j(t)*e3_j(t))*Derivative(uz_j(t), (t, 2)) - 2*(e0_j(t)*e3_j(t) - e1_j(t)*e2_j(t))*Derivative(uy_j(t), (t, 2)) - 2*(2*Derivative(e0_i(t), t)*Derivative(e2_i(t), t) + 2*Derivative(e1_i(t), t)*Derivative(e3_i(t), t))*uz_i(t) + 2*(2*Derivative(e0_i(t), t)*Derivative(e3_i(t), t) - 2*Derivative(e1_i(t), t)*Derivative(e2_i(t), t))*uy_i(t) + 2*(2*Derivative(e0_j(t), t)*Derivative(e2_j(t), t) + 2*Derivative(e1_j(t), t)*Derivative(e3_j(t), t))*uz_j(t) - 2*(2*Derivative(e0_j(t), t)*Derivative(e3_j(t), t) - 2*Derivative(e1_j(t), t)*Derivative(e2_j(t), t))*uy_j(t) - 4*(e0_i(t)*Derivative(e0_i(t), t) + e1_i(t)*Derivative(e1_i(t), t) - e2_i(t)*Derivative(e2_i(t), t) - e3_i(t)*Derivative(e3_i(t), t))*Derivative(ux_i(t), t) - 4*(e0_i(t)*Derivative(e2_i(t), t) + e1_i(t)*Derivative(e3_i(t), t) + e2_i(t)*Derivative(e0_i(t), t) + e3_i(t)*Derivative

In [23]:
dij.diff('t')

Matrix([
[Derivative(x_i(t), t)],
[Derivative(y_i(t), t)],
[Derivative(z_i(t), t)]])

In [ ]:
class spehrical_constraint(object):
    
    # Number of Scalar Constraint Equations
    nc  = 3
    
    def __init__(self):
        pass
    
    def construct(self, obj):
        
        e0_i, 
        
        pos_level_equation = Matrix([[ux_i*(e0_i**2 + e1_i**2 - e2_i**2 - e3_i**2) - ux_j*(e0_j**2 + e1_j**2 - e2_j**2 - e3_j**2) + uy_i*(-2*e0_i*e3_i + 2*e1_i*e2_i) - uy_j*(-2*e0_j*e3_j + 2*e1_j*e2_j) + uz_i*(2*e0_i*e2_i + 2*e1_i*e3_i) - uz_j*(2*e0_j*e2_j + 2*e1_j*e3_j) + x_i - x_j], 
                                     [ux_i*(2*e0_i*e3_i + 2*e1_i*e2_i) - ux_j*(2*e0_j*e3_j + 2*e1_j*e2_j) + uy_i*(e0_i**2 - e1_i**2 + e2_i**2 - e3_i**2) - uy_j*(e0_j**2 - e1_j**2 + e2_j**2 - e3_j**2) + uz_i*(-2*e0_i*e1_i + 2*e2_i*e3_i) - uz_j*(-2*e0_j*e1_j + 2*e2_j*e3_j) + y_i - y_j], 
                                     [ux_i*(-2*e0_i*e2_i + 2*e1_i*e3_i) - ux_j*(-2*e0_j*e2_j + 2*e1_j*e3_j) + uy_i*(2*e0_i*e1_i + 2*e2_i*e3_i) - uy_j*(2*e0_j*e1_j + 2*e2_j*e3_j) + uz_i*(e0_i**2 - e1_i**2 - e2_i**2 + e3_i**2) - uz_j*(e0_j**2 - e1_j**2 - e2_j**2 + e3_j**2) + z_i - z_j]])
        
        vel_level_equation = zero_matrix(3,1)
        acc_level_equation = B(obj.Pdi,obj.ui_bar)*obj.Pdi\
                            -B(obj.Pdj,obj.uj_bar)*obj.Pdj
            
        jacobian = ([ I,  obj.Bui] , [-I, -obj.Buj])
        
        obj._pos_level_equations.append(pos_level_equation)
        obj._vel_level_equations.append(vel_level_equation)
        obj._acc_level_equations.append(acc_level_equation)
        
        obj._jacobian_i.append(jacobian[0])
        obj._jacobian_j.append(jacobian[1])